In [21]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
import math
import cv2
import os
import keras.backend as K
from google.colab import drive
from keras import backend as K
from keras.layers import Layer
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input, Conv2D, MaxPool2D, AveragePooling2D, GlobalAveragePooling2D, Flatten, Dense, Dropout, concatenate
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report, confusion_matrix

In [22]:
# use GPU

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [23]:
# import dataset

drive.mount('/content/drive')
os.chdir("/content/drive/My Drive")

labels = ['Baked Potato', 'Burger', 'Crispy Chicken', 'Donut', 'Fries',
          'Hot Dog', 'Pizza', 'Sandwich', 'Taco', 'Taquito']
img_size = 150
def get_data(data_dir):
    data = []
    for label in labels:
        path = os.path.join(data_dir, label)
        class_num = labels.index(label)
        for img in os.listdir(path):
            try:
                img_arr = cv2.imread(os.path.join(path, img))[...,::-1]
                resized_arr = cv2.resize(img_arr, (img_size, img_size))
                data.append([resized_arr, class_num])
            except Exception as e:
                print(e)
    return np.array(data)

train = get_data("/content/drive/MyDrive/Klasifikasi Makanan Cepat Saji/Training")
test = get_data("/content/drive/MyDrive/Klasifikasi Makanan Cepat Saji/Testing")

X_train = []
y_train = []
X_test = []
y_test = []

for feature, label in train:
  X_train.append(feature)
  y_train.append(label)

for feature, label in test:
  X_test.append(feature)
  y_test.append(label)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-23-2b4332763041>:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(data)


In [24]:
# normalize data

X_train = np.array(X_train) / 255
X_train = X_train.reshape(-1, img_size, img_size, 3)
X_train = X_train.astype('float32')
y_train = np.array(y_train)
y_train = to_categorical(y_train)

X_test = np.array(X_test) / 255
X_test = X_test.reshape(-1, img_size, img_size, 3)
X_test = X_test.astype('float32')
y_test = np.array(y_test)
y_test = to_categorical(y_test)

In [25]:
X_train.shape

(2400, 150, 150, 3)

In [26]:
y_train.shape

(2400, 10)

In [27]:
# inception module

def inception_module(x,filters_a,
                     filters_b1,filters_b2,
                     filters_c1,filters_c2,
                     filters_d1,filters_d2,
                     filters_pool_proj,name=None):
  conv_a = Conv2D(filters_a, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
  conv_b = Conv2D(filters_b1, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
  conv_b = Conv2D(filters_b2, (3, 3), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(conv_b)
  conv_c = Conv2D(filters_c1, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
  conv_c = Conv2D(filters_c2, (5, 5), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(conv_c)
  conv_d = Conv2D(filters_d1, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
  conv_d = Conv2D(filters_d2, (7, 7), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(conv_d)
  pool_proj = MaxPool2D((3, 3), strides=(1, 1), padding='same')(x)
  pool_proj = Conv2D(filters_pool_proj, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(pool_proj)
  output = concatenate([conv_a, conv_b, conv_c, conv_d, pool_proj], axis=3, name=name)
  return output

kernel_init = keras.initializers.glorot_uniform()
bias_init = keras.initializers.Constant(value=0.2)

In [28]:
# build model

inp = Input(shape=(150, 150, 3))
x = Conv2D(32, (3, 3), padding='same', strides=(1, 1), activation='relu')(inp)
x = MaxPool2D((3, 3), padding='same', strides=(2, 2))(x)
x = inception_module(x,filters_a=32,
                     filters_b1=32,filters_b2=48,
                     filters_c1=32,filters_c2=48,
                     filters_d1=32,filters_d2=48,
                     filters_pool_proj=64)
x = inception_module(x,filters_a=32,
                     filters_b1=32,filters_b2=64,
                     filters_c1=32,filters_c2=64,
                     filters_d1=32,filters_d2=64,
                     filters_pool_proj=80)
x = MaxPool2D((3, 3), padding='same', strides=(2, 2))(x)
x = inception_module(x,filters_a=32,
                     filters_b1=32,filters_b2=80,
                     filters_c1=32,filters_c2=80,
                     filters_d1=32,filters_d2=80,
                     filters_pool_proj=96)
x = inception_module(x,filters_a=32,
                     filters_b1=32,filters_b2=96,
                     filters_c1=32,filters_c2=96,
                     filters_d1=32,filters_d2=96,
                     filters_pool_proj=128)
x = GlobalAveragePooling2D()(x)
x = Dropout(0.2)(x)
x = Dense(10, activation='softmax')(x)
model = Model(inp, x)

/usr/local/lib/python3.10/dist-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


In [29]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 150, 150, 3)]        0         []                            
                                                                                                  
 conv2d_33 (Conv2D)          (None, 150, 150, 32)         896       ['input_2[0][0]']             
                                                                                                  
 max_pooling2d_6 (MaxPoolin  (None, 75, 75, 32)           0         ['conv2d_33[0][0]']           
 g2D)                                                                                             
                                                                                                  
 conv2d_35 (Conv2D)          (None, 75, 75, 32)           1056      ['max_pooling2d_6[0][0]'

In [30]:
batch_size = 60
epochs = 300
data_augmentation = True
initial_lrate = 0.0001

def decay(epoch, steps=100):
  initial_lrate = 0.0001
  drop = 0.96
  epochs_drop = 8
  lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
  return lrate

opt = Adam(lr=initial_lrate)
lr_sc = LearningRateScheduler(decay, verbose=1)
model.compile(optimizer=opt,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [31]:
H = model.fit(np.array(X_train), np.array(y_train), validation_data=(X_test, y_test), epochs=epochs, batch_size=batch_size, callbacks=[lr_sc])


Epoch 1: LearningRateScheduler setting learning rate to 0.0001.
Epoch 1/300
40/40 [==============================] - 28s 449ms/step - loss: 2.3454 - accuracy: 0.1133 - val_loss: 2.2835 - val_accuracy: 0.1583 - lr: 1.0000e-04

Epoch 2: LearningRateScheduler setting learning rate to 0.0001.
Epoch 2/300
40/40 [==============================] - 17s 429ms/step - loss: 2.2979 - accuracy: 0.1192 - val_loss: 2.2362 - val_accuracy: 0.1733 - lr: 1.0000e-04

Epoch 3: LearningRateScheduler setting learning rate to 0.0001.
Epoch 3/300
40/40 [==============================] - 17s 434ms/step - loss: 2.2566 - accuracy: 0.1646 - val_loss: 2.1954 - val_accuracy: 0.1883 - lr: 1.0000e-04

Epoch 4: LearningRateScheduler setting learning rate to 0.0001.
Epoch 4/300
40/40 [==============================] - 18s 444ms/step - loss: 2.2379 - accuracy: 0.1617 - val_loss: 2.2183 - val_accuracy: 0.1483 - lr: 1.0000e-04

Epoch 5: LearningRateScheduler setting learning rate to 0.0001.
Epoch 5/300
40/40 [============

KeyboardInterrupt: 

In [ ]:
acc = H.history['accuracy']
val_acc = H.history['val_accuracy']
loss = H.history['loss']
val_loss = H.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(15, 15))
plt.subplot(2, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

predictions_x=model.predict(X_test)
classes_x=np.argmax(predictions_x,axis=1)

y_test=np.argmax(y_test,axis=1)

print(classification_report(y_test, classes_x, target_names = ['Baked Potato', 'Burger', 'Crispy Chicken', 'Donut', 'Fries',
                                                               'Hot Dog', 'Pizza', 'Sandwich', 'Taco', 'Taquito']))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

# Misalnya, Anda sudah memiliki confusion matrix
cm = confusion_matrix(y_test, classes_x)

# Membuat heatmap dari confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
plt.title('Confusion Matrix')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()

In [ ]:
from prettytable import PrettyTable

# Data akurasi pelatihan dan pengujian
train_accuracy = H.history['accuracy'][-1] * 100  # Konversi ke persen
test_accuracy = H.history['val_accuracy'][-1] * 100  # Konversi ke persen

# Membuat tabel
table = PrettyTable()
table.field_names = ["Metric", "Value"]
table.add_row(["Train Accuracy", f"{train_accuracy:.2f}%"])
table.add_row(["Test Accuracy", f"{test_accuracy:.2f}%"])

# Mencetak tabel
print(table)

In [ ]:
def show_images(images, labels, class_names, model=None, GRID=[5,6], SIZE=(25,25)):

    # Plot Configurations
    n_rows, n_cols = GRID
    n_images = n_cols * n_rows
    plt.figure(figsize=SIZE)

    # Iterate Through the data
    i = 1
    for id in np.random.randint(len(images), size=n_images):

        # Select data Randomly
        image, label = tf.expand_dims(images[id], axis=0), class_names[int(labels[id])]

        # Show Image
        plt.subplot(n_rows, n_cols, i)
        plt.imshow(image[0])

        # make Prediction
        if model is not None:
            prediction = model.predict(image)[0]
            score = np.round(max(prediction), 2)
            pred = class_names[np.argmax(prediction)]
            title = f"True : {label}\nPred : {pred}"
        else:
            title = label

        plt.title(title)
        plt.axis('off')

        # Break Loop
        i += 1
        if i > n_images:
            break

    # Show Final Plot
    plt.show()

show_images(images=X_test, labels=y_test, class_names=labels, model=model)